# Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [64]:
import pandas as pd

df = pd.read_csv('churn.csv')

In [65]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [66]:
X = df.drop(columns=['Churn', 'customerID'])
X.shape

(7043, 19)

In [67]:
X['TotalCharges'] = X['TotalCharges'].replace(' ', '20.2')

In [68]:
X['TotalCharges'] = X['TotalCharges'].astype('float64')

In [69]:
X.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
dtype: object

In [70]:
y = df['Churn']

In [71]:
X = pd.get_dummies(X)

In [72]:
X.isnull().sum().sum()

0

In [73]:
y = y.replace({'No':0,'Yes':1})

In [74]:
X = X.values
y = y.values

In [75]:
X.shape, y.shape

((7043, 45), (7043,))

In [76]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5634, 45), (1409, 45), (5634,), (1409,))

In [77]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Keras Baseline Modeling

In [78]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [79]:
model = Sequential()

model.add(Dense(64, activation='sigmoid', input_dim=45))
model.add(Dense(64, activation='sigmoid'))

model.add(Dense(1, activation='sigmoid'))

In [80]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [83]:
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
5634/5634 [==============================] - 2s 327us/step - loss: 0.2379 - acc: 0.8900
Epoch 2/20
5634/5634 [==============================] - 2s 362us/step - loss: 0.2395 - acc: 0.8939
Epoch 3/20
5634/5634 [==============================] - 2s 403us/step - loss: 0.2382 - acc: 0.8926
Epoch 4/20
5634/5634 [==============================] - 2s 412us/step - loss: 0.2372 - acc: 0.8931
Epoch 5/20
5634/5634 [==============================] - 2s 401us/step - loss: 0.2359 - acc: 0.8942
Epoch 6/20
5634/5634 [==============================] - 2s 345us/step - loss: 0.2364 - acc: 0.8940
Epoch 7/20
5634/5634 [==============================] - 2s 359us/step - loss: 0.2355 - acc: 0.8947
Epoch 8/20
5634/5634 [==============================] - 2s 353us/step - loss: 0.2326 - acc: 0.8960
Epoch 9/20
5634/5634 [==============================] - 2s 350us/step - loss: 0.2335 - acc: 0.8979
Epoch 10/20
5634/5634 [==============================] - 2s 358us/step - loss: 0.2321 - acc: 0.8919
Epoch 11/

In [82]:
scores = model.evaluate(X_test, y_test)
print(f"Test Acc: {scores[1]*100}")

1409/1409 [==============================] - 1s 931us/step
Test Acc: 76.65010645848119


## Tuning

In [60]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

def create_model():
    model = Sequential()
    model.add(Dense(64, activation='sigmoid', input_dim=45))
    model.add(Dense(64, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=1)

In [62]:
param_grid = {
    'batch_size':[20, 32, 40, 60, 100],
    'epochs':[20]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)

/anaconda3/envs/U4-S2-NNF/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/20
3756/3756 [==============================] - 2s 510us/step - loss: 0.4811 - acc: 0.7689
Epoch 2/20
3756/3756 [==============================] - 1s 145us/step - loss: 0.4384 - acc: 0.7942
Epoch 3/20
3756/3756 [==============================] - 1s 156us/step - loss: 0.4356 - acc: 0.7926
Epoch 4/20
3756/3756 [==============================] - 1s 144us/step - loss: 0.4336 - acc: 0.7921
Epoch 5/20
3756/3756 [==============================] - 1s 145us/step - loss: 0.4301 - acc: 0.7971
Epoch 6/20
3756/3756 [==============================] - 1s 145us/step - loss: 0.4304 - acc: 0.7947
Epoch 7/20
3756/3756 [==============================] - 1s 145us/step - loss: 0.4293 - acc: 0.7939
Epoch 8/20
3756/3756 [==============================] - 1s 151us/step - loss: 0.4286 - acc: 0.7966
Epoch 9/20
3756/3756 [==============================] - 1s 149us/step - loss: 0.4283 - acc: 0.7990
Epoch 10/20
3756/3756 [==============================] - 1s 147us/step - loss: 0.4286 - acc: 0.7977
Epoch 11/

In [63]:
print(f"Best: {grid_result.best_score_} with {grid_result.best_params_}")
print("------------")

means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]

for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev}, with: {param}")

Best: 0.8001419965959332 with {'batch_size': 20, 'epochs': 20}
------------
Means: 0.8001419965959332, Stdev: 0.005186953111891818, with: {'batch_size': 20, 'epochs': 20}
Means: 0.7946396877167281, Stdev: 0.003514197735500675, with: {'batch_size': 32, 'epochs': 20}
Means: 0.7985445511099883, Stdev: 0.003920966498154218, with: {'batch_size': 40, 'epochs': 20}
Means: 0.7969471069570547, Stdev: 0.00435493372830877, with: {'batch_size': 60, 'epochs': 20}
Means: 0.7944621953466576, Stdev: 0.005751453678974827, with: {'batch_size': 100, 'epochs': 20}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?